In [1]:
%reset -fs

import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation

# pd.set_option('max_columns', 100)
pd.set_option("display.max_colwidth", 1)

In [2]:
df = pd.read_csv('tweets_01-08-2021.csv', encoding='utf-8')

In [3]:
len(df)

56571

In [4]:
df.head(30)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created our economic problems.,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,"I was thrilled to be back in the Great city of Charlotte, North Carolina with thousands of hardworking American Patriots who love our Country, cherish our values, respect our laws, and always put AMERICA FIRST! Thank you for a wonderful evening!! #KAG2020 https://t.co/dNJZfRsl9y",f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance court obtained by CBS News questions where there will be further disciplinary action and cho…,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,"The Unsolicited Mail In Ballot Scam is a major threat to our Democracy, &amp; the Democrats know it. Almost all recent elections using this system, even though much smaller &amp; with far fewer Ballots to count, have ended up being a disaster. Large numbers of missing Ballots &amp; Fraud!",f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT @MZHemingway: Very friendly telling of events here about Comey's apparent leaking to compliant media. If you read those articles and tho…,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,RT @WhiteHouse: President @realDonaldTrump announced historic steps to protect the Constitutional right to pray in public schools! https://…,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,Getting a little exercise this morning! https://t.co/fyAAcbhbgk,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
7,1319501865625784320,https://t.co/4qwCKQOiOw,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14,f
8,1319500520126664705,https://t.co/VlEu8yyovv,f,f,Twitter for iPhone,153446,20275,2020-10-23 04:46:53,f
9,1319500501269041154,https://t.co/z5CRqHO8vg,f,f,Twitter for iPhone,102150,14815,2020-10-23 04:46:49,f


In [5]:
df.groupby('isFlagged').count()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date
isFlagged,,,,,,,,
f,56267,56267,56267,56267,56267,56267,56267,56267
t,304,304,304,304,304,304,304,304


In [6]:
# Text preprocessing steps - remove numbers, URL, captial letters and punctuation
import re
import string

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub(r'[^\w\s]', ' ',str(x).lower())
no_url = lambda x: re.sub(r'http\S+', '', x)
clean = lambda x: x.replace('\n', '')
english_only = lambda string: re.sub(r'[^\x00-\x7f]', "", string)

df['text'] = df.text.map(alphanumeric).map(no_url).map(punc_lower).map(clean).map(english_only)
df.head(50)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,republicans and democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,i was thrilled to be back in the great city of charlotte north carolina with thousands of hardworking american patriots who love our country cherish our values respect our laws and always put america first thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,rt cbs_herridge read letter to surveillance court obtained by cbs news questions where there will be further disciplinary action and cho,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,the unsolicited mail in ballot scam is a major threat to our democracy amp the democrats know it almost all recent elections using this system even though much smaller amp with far fewer ballots to count have ended up being a disaster large numbers of missing ballots amp fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,rt mzhemingway very friendly telling of events here about comey s apparent leaking to compliant media if you read those articles and tho,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,rt whitehouse president realdonaldtrump announced historic steps to protect the constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,getting a little exercise this morning,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
7,1319501865625784320,,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14,f
8,1319500520126664705,,f,f,Twitter for iPhone,153446,20275,2020-10-23 04:46:53,f
9,1319500501269041154,,f,f,Twitter for iPhone,102150,14815,2020-10-23 04:46:49,f


In [7]:
df['text'] = df['text'].apply(lambda x: x.strip()).replace('', np.nan) # fill empty cells w/ Null values

In [8]:
df.head(50)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,republicans and democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,i was thrilled to be back in the great city of charlotte north carolina with thousands of hardworking american patriots who love our country cherish our values respect our laws and always put america first thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,rt cbs_herridge read letter to surveillance court obtained by cbs news questions where there will be further disciplinary action and cho,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,the unsolicited mail in ballot scam is a major threat to our democracy amp the democrats know it almost all recent elections using this system even though much smaller amp with far fewer ballots to count have ended up being a disaster large numbers of missing ballots amp fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,rt mzhemingway very friendly telling of events here about comey s apparent leaking to compliant media if you read those articles and tho,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,rt whitehouse president realdonaldtrump announced historic steps to protect the constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,getting a little exercise this morning,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
7,1319501865625784320,NaN,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14,f
8,1319500520126664705,NaN,f,f,Twitter for iPhone,153446,20275,2020-10-23 04:46:53,f
9,1319500501269041154,NaN,f,f,Twitter for iPhone,102150,14815,2020-10-23 04:46:49,f


In [9]:
df = df.dropna() #drop null rows
df.head(50)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,republicans and democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,i was thrilled to be back in the great city of charlotte north carolina with thousands of hardworking american patriots who love our country cherish our values respect our laws and always put america first thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,rt cbs_herridge read letter to surveillance court obtained by cbs news questions where there will be further disciplinary action and cho,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,the unsolicited mail in ballot scam is a major threat to our democracy amp the democrats know it almost all recent elections using this system even though much smaller amp with far fewer ballots to count have ended up being a disaster large numbers of missing ballots amp fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,rt mzhemingway very friendly telling of events here about comey s apparent leaking to compliant media if you read those articles and tho,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,rt whitehouse president realdonaldtrump announced historic steps to protect the constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,getting a little exercise this morning,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
11,1319496349092511744,rt erictrump,t,f,Twitter for iPhone,0,8921,2020-10-23 04:30:19,f
12,1236502342121541632,rt gopchairwoman the economic boom continues jobs added in february jobs added since realdonaldtrump was electedunemploym,t,f,Twitter for iPhone,0,8681,2020-03-08 04:01:46,f
13,1225835449379258368,rt tomfitton vindman s behavior is a scandal he should be removed from the realdonaldtrump white house asap to protect our foreign poli,t,f,Twitter for iPhone,0,7679,2020-02-07 17:35:20,f


In [10]:
from datetime import datetime as dt

In [11]:
df["date"] = pd.to_datetime(df.date , format="%Y/%m/%d %H:%M:%S") #create date_time column

In [ ]:
# for tweet in df['text'].sample(10).values:
#     print(tweet)

In [12]:
corpus = df['text']

In [13]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import nltk

lemmatizer = WordNetLemmatizer()

In [14]:

words = ['gave','went','going','dating']
for word in words:
    print (word +"-->"+WordNetLemmatizer().lemmatize(word,'v'))

gave-->give
went-->go
going-->go
dating-->date


In [18]:
for tweet in corpus:
    for word in tweet:
        word1 = WordNetLemmatizer().lemmatize(word,'v')
        df['tweet_lm'] = word1
    print(tweet)

republicans and democrats have both created our economic problems
i was thrilled to be back in the great city of charlotte  north carolina with thousands of hardworking american patriots who love our country  cherish our values  respect our laws  and always put america first  thank you for a wonderful evening
rt  cbs_herridge  read  letter to surveillance court obtained by cbs news questions where there will be further disciplinary action and cho
the unsolicited mail in ballot scam is a major threat to our democracy   amp  the democrats know it  almost all recent elections using this system  even though much smaller  amp   with far fewer ballots to count  have ended up being a disaster  large numbers of missing ballots  amp  fraud
rt  mzhemingway  very friendly telling of events here about comey s apparent leaking to compliant media  if you read those articles and tho
rt  whitehouse  president  realdonaldtrump announced historic steps to protect the constitutional right to pray in publ

foxnews   quinnipiacpoll  abc wapo  nbc wsj were so inaccurate with their polls on me  that it really is tampering with an election  they were so far off in their polling  and in their attempt to suppress   that they should be called out for election interference
rt  dbongino  this
pleased to announce that  kloeffler  amp   sendavidperdue have just joined our great  stopthesteal group of senators  they will fight the ridiculous electoral college certification of biden  how do you certify numbers that have now proven to be wrong and  in many cases  fraudulent
if joe biden were president  you wouldn t have the vaccine for another four years  nor would the  us_fda have ever approved it so quickly  the bureaucracy would have destroyed millions of lives
the  us_fda and the democrats didn t want to have me get a vaccine win  prior to the election  so instead it came out five days later   as i ve said all along
who will be running for governor of the great state of ohio  will be hotly contest

a total fraud  statehouse republicans  proud  strong and honest  will never let this travesty stand
rt  mtaibbi   the least curious people in the country right now appear to be the credentialed news media  a situation normally unique to ti
rt  sohrabahmari  it s ok  the post has weathered the vicissitudes of more than two centuries of american history  we will endure this  too
looks like they are setting up a big  voter dump  against the republican candidates  waiting to see how many votes they need
highly respected ken starr   pennsylvania s three day extension of the mail in ballot deadline is a constitutional travesty   legal scholars agree
will use these charts in court case  the election was a hoax  thank you justin
will go down much further  weekend daytime even worse  dead  they still don t get it  fantastic alternatives  the late great roger ailes is seriously missed  but i still won legal votes by a lot
rt  steveguest     joe biden brags about having  the most extensive and in

giuliani  the case for election fraud being made by americans  via  breitbartnews
report  biden s thanksgiving day address gets just   views online  a record low  observers say a candidate with         votes would get many more online viewers  numbers don t lie  or add up   oann
rt  jim_jordan  don t lockdown the country don t impose curfews don t close schools  let americans decide for themselves  and celebra
rt  realdonaldtrump  just saw the vote tabulations  there is no way biden got       votes    this was a    rigged election
these scoundrels are only toying with the  sendavidperdue  a great guy  vote  just didn t want to announce quite yet  they ve got as many ballots as are necessary  rigged election
that s why mike was number one in his class at west point
rt  katrinapierson      trump
rt  realdonaldtrump  thank you megyn
thanks dan  big win for us in texas
news conference on reduced drug prices at     p m
rt  realdonaldtrump  a must read  impossible for biden to have overcome 

pennsylvania ballot  mistakes  made  in my opinion  on purpose by the democrat governor  puts republicans  in particular  seanparnellusa  at great risk  corrupt politics in pennsylvania must be investigated immediately  by the way  philadelphia is out of control   thejusticedept
to all of those who have asked  i will not be going to the inauguration on january
rt  arthurschwartz  texas lt  gov  dan patrick offers    million reward for evidence of voter fraud
this says it all
i have reversed the ridiculous decision to cancel wreaths across america at arlington national cemetery  it will now go on
rt  txelectionlaw  having practiced law for almost   decades as a member of the state bar of texas  i ve never known fellow tx lawyer  sidn
the recent statement by chris krebs on the security of the   election was highly inaccurate  in that there were massive improprieties and fraud   including dead people voting  poll watchers not allowed into polling locations   glitches  in the voting machin

...

rt  realdonaldtrump  i was very surprised  amp  disappointed that senator joe manchin of west virginia voted against me on the democrat s total
rt  realdonaldtrump      always continue  every republican senator except romney  many highly religious people  all very smart  voted again
elizabeth  pocahontas  warren singlehandedly destroyed the bernie sanders campaign by stripping voters away from his count on super t  he lost states that he easily would have won if she had dropped out   days earlier  the dnc is doing it to bernie again  will he ever get angry
animals that must be hit hard
rt  realdonaldtrump  border wall is going up fast  over   miles already built    miles by early next year  or sooner  dems want open bo
rt     air force one landing in virginia
yes
rt  realdonaldtrump  thank you   maga
rt  gop
rt  gop   we ve known this has been a waste of time and a sham from the beginning  and america is ready to move on  i think they ve shown t
was given a horrendous report by his sup

rt  gop  just in   realdonaldtrump donates his quarterly salary to  hhsgov to help fight coronavirus  thank you  president trump
another fake book by two third rate washington post reporters  has already proven to be inaccurately reported  to their great embarrassment  all for the purpose of demeaning and belittling a president who is getting great things done for our country  at a record clip  thank you
the obama biden administration is the most corrupt administration in the history of our country
seriously failed presidential candidate  senator elizabeth  pocahontas  warren  just introduced an amendment on the renaming of many of our legendary military bases from which we trained to win two world wars  hopefully our great republican senators won t fall for this
rt  mitchellvii  flynn attorney sidney powell  the documents that are yet to be released  are even more stunning   video
polls are showing tremendous disapproval of lamestream media coverage of the virus crisis  the fake news 

president trump is a ratings hit  since reviving the daily white house briefing mr  trump and his coronavirus updates have attracted an average audience of     million on cable news  roughly the viewership of the season finale of  the bachelor   numbers are continuing to rise
the radical left  do nothing democrats  don t want justice when pushing the impeachment hoax  they only want to destabilize the republican party so they can do better in the   election   amp  that includes the house  amp  senate  they are playing with the people by taking it this far
more fake news  this was photoshopped  obviously  but the wind was strong and the hair looks good  anything to demean
thank you
the biggest loser tonight  by far  is mini mike bloomberg  his  political  consultants took him for a ride     million washed down the drain  and he got nothing for it but the nickname mini mike  and the complete destruction of his reputation  way to go mike
rt  realwayneroot  hard to believe cavuto is this f

i promised to restore hope in america  that includes the least among us  together  let s keep america great  text trump to   if you liked our super bowl ad
rt     here s how president donald trump thanked  repandybiggsaz  and  repdlesko of arizona
rt  whitehouse  live  members of the coronavirus task force hold a press briefing
rt  whitehouse   we re working to finalize an executive order that will encourage police departments nationwide to meet the most current pr
rt  whitehouse  president  realdonaldtrump announces four steps to build safety  opportunity  and dignity for all americans and build on vi
u s  cancer death rate lowest in recorded history  a lot of good news coming out of this administration
breaking news  the fifth circuit court of appeals just reversed a lower court decision  amp  gave us the go ahead to build one of the largest sections of the desperately needed southern border wall  four billion dollars  entire wall is under construction or getting ready to start
hope 

KeyboardInterrupt: 

In [21]:
df.head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged,tweet_lm
0,98454970654916608,republicans and democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f,p
1,1234653427789070336,i was thrilled to be back in the great city of charlotte north carolina with thousands of hardworking american patriots who love our country cherish our values respect our laws and always put america first thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f,p
2,1218010753434820614,rt cbs_herridge read letter to surveillance court obtained by cbs news questions where there will be further disciplinary action and cho,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f,p
3,1304875170860015617,the unsolicited mail in ballot scam is a major threat to our democracy amp the democrats know it almost all recent elections using this system even though much smaller amp with far fewer ballots to count have ended up being a disaster large numbers of missing ballots amp fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f,p
4,1218159531554897920,rt mzhemingway very friendly telling of events here about comey s apparent leaking to compliant media if you read those articles and tho,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f,p


In [ ]:
# def pos_tagger(nltk_tag):
#     if nltk_tag.startswith('J'):
#         return wordnet.ADJ
#     elif nltk_tag.startswith('V'):
#         return wordnet.VERB
#     elif nltk_tag.startswith('N'):
#         return wordnet.NOUN
#     elif nltk_tag.startswith('R'):
#         return wordnet.ADV
#     else:         
#         return None

In [ ]:
# tokenize the sentence and find the POS tag for each token
# pos_tagged = nltk.pos_tagger(nltk.word_tokenize(corpus)) 

In [ ]:
# df['text'] =  df['text'].apply(lambda sentence: [lemmatizer.lemmatize for w in word_tokenize(corpus)])

In [ ]:
# stop_words = ENGLISH_STOP_WORDS

In [ ]:
tfidf = TfidfVectorizer(max_df=0.8,min_df=5)
tweet_word_matrix = tfidf.fit_transform(corpus)
vocab = tfidf.get_feature_names_out()

In [ ]:
tweet_word_matrix.shape

In [ ]:
nmf = NMF(n_components=30)

nmf.fit(tweet_word_matrix)

In [ ]:
tweet_topic_matrix = nmf.transform(tweet_word_matrix)

In [ ]:
tweet_topic_matrix_df = pd.DataFrame(tweet_topic_matrix).add_prefix('topic_')
tweet_topic_matrix_df['tweets'] = df['text']
tweet_topic_matrix_df.head()

In [ ]:
word_topic_matrix_df = pd.DataFrame(nmf.components_, columns=vocab).T.add_prefix('topic_')
# word_topic_matrix_df.tail(5)

In [ ]:
# for tweet in tweet_topic_matrix_df.sort_values(by='topic_0', ascending=False).head(10)['tweets'].values:
#     print(tweet)
#     print()

In [ ]:
word_topic_matrix_df.sort_values(by='topic_0', ascending=False).head(10)

In [ ]:
def top_tweets(tweet_topic_matrix_df, topic, n_tweets):
    return (tweet_topic_matrix_df
            .sort_values(by=topic, ascending=False)
            .head(n_tweets)['tweets']
            .values)

def top_words(word_topic_matrix_df, topic, n_words):
    return (word_topic_matrix_df
            .sort_values(by=topic, ascending=False)
            .head(n_words))[topic]

In [ ]:
# for tweet in top_tweets(tweet_topic_matrix_df, 'topic_1', 15):
#     print(tweet)
#     print()

In [ ]:
top_words(word_topic_matrix_df, 'topic_0', 10)

In [ ]:
top_words(word_topic_matrix_df, 'topic_1', 10)

In [ ]:
top_words(word_topic_matrix_df, 'topic_2', 10)

In [ ]:
top_words(word_topic_matrix_df, 'topic_3', 10)

In [ ]:
# for tweet in top_tweets(tweet_topic_matrix_df, 'topic_2', 5):
#     print(tweet)

In [ ]:
# for tweet in top_tweets(tweet_topic_matrix_df, 'topic_2', 5):
#     print(tweet)
#     print()

In [ ]:
df_before = df[(df['date'] <= "2017-01-19")]
df_before

In [ ]:
df_after = df[(df['date'] > "2017-01-20")]
df_after

In [ ]:
df_before2 = df_before['text']
df_before2.iloc[0]

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()


In [ ]:
for i in 

In [ ]:
score = analyzer.polarity_scores(df_before2.iloc[0])

In [ ]:
print(score)